**Importy**

In [30]:
import regex
from csv import writer
import pandas as pd 
import numpy as np
from stop_words import get_stop_words

In [31]:
import majka
from majka import Majka
morph = Majka('../data/w-lt.sk.fsa')

morph.flags |= majka.ADD_DIACRITICS  # find word forms with diacritics
morph.flags |= majka.DISALLOW_LOWERCASE  # do not enable to find lowercase variants
morph.flags |= majka.IGNORE_CASE  # ignore the word case whatsoever
morph.flags = 0  # unset all flags

morph.tags = False  # return just the lemma, do not process the tags
# morph.tags = True  # turn tag processing back on (default)

morph.first_only = True  # return only the first entry
# morph.first_only = False  # return all entries (default)

In [32]:
from corpy.morphodita import Tagger
tagger = Tagger("../data/slovak-morfflex-pdt-170914.tagger")

INFO:corpy.morphodita:Loading tagger.


In [33]:
import nltk
nltk.download('punkt')
# nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rusna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Premenne**

In [34]:
# file s jednou page
debug_filepath = "../data/test_data_1page.xml"
# cely file
input_filepath = "../data/skwiki-latest-pages-articles.xml"
output_filepath = "../data/anchors.csv"

**Funkcie**

In [35]:
# otvorenie suboru, zadefinovanie hlavicky csv suboru a nasledne citanie s pracovanie po riadkoch
def open_read(input_filepath,output_filepath,process):
    with open(input_filepath, encoding='UTF-8') as input_file ,open(output_filepath, 'w+', encoding='UTF-8' ) as output_file:
        csv_writer = writer(output_file, delimiter='|')
        csv_writer.writerow(["Link", "Alt_Text"])
        for line in input_file:
            process(line,csv_writer)

In [36]:
# využitie regexu na nájdene všetkých zhôd v riadku a zapísanie do csv súboru. 
def process(line,csv_writer):
    regex_patern = '\[\[([\p{L}\p{Pc}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})= ]+)\|?([\p{L}\p{Pc}\p{Po}\p{Pi}\p{Pf}\p{Ps}\p{Pd}\p{Sk}\p{So}\p{C}\p{N}\p{Z}})= ]+)?\]\]'
    #\[\[([^|\]\]]+)[^\]\]]*\]\]   #\[\[([\p{L}:\s\{\}\.\/\(\)]+)\|?([^\]]*)?\]\]
    matches = regex.findall(regex_patern, line) 
    for match in matches:
        list_matche = list(filter(None, match))
        
        
        csv_writer.writerow(list_matche)

In [37]:
# funkcia pre upravu textu
def replace(text, regex_pattern, substitution):
    if not isinstance(text, str):
        return text
    return regex.sub(regex_pattern,substitution,text)

In [38]:
# funkcia pre tokenizovanie textu. Text -> Vety -> Slova (Tokeny) vyuzitim nltk 
def tokenizer_modded(text):
    sentences = nltk.sent_tokenize(text)
    tokens = []
    for sent in sentences:
        tokens.append(nltk.word_tokenize(sent))
    flatten_list = [j for sub in tokens for j in sub] 
    result = [token.lower() for token in flatten_list if token not in ".,?!...-"]
    
    return result

In [129]:
# lematizovanie textu s odstranenim stop slov. Vyuziva sa majka so Slovenskym Slovnikom
def lematizer_modded(text):
    stop_words = get_stop_words('sk')

    result = []
    for word in text:
        morph_word = morph.find(word)
#         Aktualne slova ktore nemaju lemu, budu mat povodny tvar.
        if morph_word:
            result.append(morph_word[0]['lemma'])
        else:
            if hashmap_lem[word] > THRESHOLD:
                result.append(word)
            else:
                result.append(None)
    for word in result:  # odstranenie stopslov
        if word in stop_words:
            result.remove(word)
    return result

In [130]:
def fill_hashmap(text):
  
    for word in text:
        morph_word = morph.find(word)
#         print(morph_word)
        if morph_word == []:
            if not word in hashmap_lem:
                hashmap_lem[word] = 1
#                 print(hashmap_lem[word])
#                 print('if')
            else:
                hashmap_lem[word] += 1 
#                 print(hashmap_lem[word])
#                 print('else')
                

    

In [131]:
def process_data(df):
    for column in df:
        df[column] = df[column] \
        .apply(lambda x: x if x is not np.NaN else None) \
        .apply(replace,regex_pattern = r"[\\\/\(\):#.\_]|({{.*}})|\d*px\png\svg\jpg",substitution =' ')\
        .apply(replace,regex_pattern = r"([ ]+)",substitution =' ')\
        
    return df

In [132]:
def tokenize_data(df):
    for column in df:
        df[column] = df[column] \
        .apply(lambda text: tokenizer_modded(text) if text is not None else None )
        
    return df

In [133]:
def lemetize_data(df):
    for column in df:
        df[column] = df[column] \
        .apply(lambda text: lematizer_modded(text) if text is not None else None )
        
    return df

In [134]:
def create_hashmap(df):
    for column in df:
        print(column)
        df[column] \
        .apply(lambda text: fill_hashmap(text) if text is not None else None )
        


**Priebeh spracovania**

In [135]:
open_read(debug_filepath,output_filepath,process)

In [136]:
# spracovanie suboru po chunkoch a postupne ukladanie vysledkov do csv
hashmap_lem = {}  
for index,chunk in enumerate(pd.read_csv(output_filepath, chunksize=15, delimiter='|')):
    

    process_data(chunk)
    tokenize_data(chunk)
    create_hashmap(chunk)
    print('chunk ------------------------------------------------------------------------------',index)    
    print(chunk)
    lemetize_data(chunk)
    
    print('chunk ------------------------------------------------------------------------------',index)    
    print(chunk)
    chunk.to_csv('../data/done.csv', mode = 'a+', index=False, encoding='UTF-8')


Link
Alt_Text
chunk ------------------------------------------------------------------------------ 0
                                                 Link  \
0                                         [slovensko]   
1               [slovenská, republika, 1939, –, 1945]   
2                                         [slovensko]   
3                                      [generálmajor]   
4                                  [ľubomír, svoboda]   
5   [dopravné, krídlo, generála, milana, rastislav...   
6   [taktické, krídlo, generálmajora, otta, smika,...   
7   [vrtuľníkové, krídlo, generálplukovníka, jána,...   
8         [protilietadlová, raketová, brigáda, nitra]   
9    [brigáda, riadenia, vzdušných, operácií, zvolen]   
10              [súbor, official-emblem-of-ossr, png]   
11           [ozbrojené, sily, slovenskej, republiky]   
12         [súbor, coat, of, arms, of, slovakia, svg]   
13  [súbor, roundel, of, slovakia, –, low, visibil...   
14                             [aero, l-39, 

Link
Alt_Text
chunk ------------------------------------------------------------------------------ 13
                           Link           Alt_Text
195        [stíhacie, lietadlo]               None
196                 [18, marec]        [18, marca]
197                      [2015]               None
198     [saab, jas, 39, gripen]  [jas, 39, gripen]
199  [foreign, military, sales]              [fms]
200                      [2016]               None
201            [spojené, štáty]              [usa]
202                     [rusko]               None
203                    [mig-35]               None
204                   [švédsko]               None
205                  [maďarsko]               None
206                     [česko]               None
207                    [poľsko]           [poľska]
208                 [taliansko]        [talianska]
209                    [grécko]           [grécka]
chunk ----------------------------------------------------------------------------

359                                       [usa]  
chunk ------------------------------------------------------------------------------ 23
                                                  Link  \
345             [súbor, None, slovak, air, force, jpg]   
346                                  [česko-slovensko]   
347                                            [česko]   
348                                      [helikoptéra]   
349                                       [mil, mi-17]   
350  [súbor, mil, mi-17, slovakia, air, force, None...   
351                                  [sovietsky, zväz]   
352                     [sikorsky, uh-60, black, hawk]   
353                     [sikorsky, uh-60, black, hawk]   
354              [súbor, slovenský, uh-60m, None, jpg]   
355                                    [spojený, štát]   
356                                [stíhací, lietadlo]   
357        [general, dynamics, f-16, fighting, falcon]   
358                  [súbor, None, f-16c, profil, 

In [91]:
# TODO
# - zmenit regex na vseobecny
# - analyza
